In [1]:
#dataset link
#https://www.kaggle.com/lena0104/classification-mit-bih/data 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# **CNN for Heart Rate Monitoring**

# Importing data

In [2]:
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train_data = pd.read_csv("mitbih_train.csv", header = None) 
train_data=pd.DataFrame(train_data)
test_data = pd.read_csv("mitbih_test.csv", header = None)
test_data=pd.DataFrame(test_data)

# Data Preprocessing

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.describe().T  

In [ ]:
sns.catplot(x = 187, kind = 'count', data = train_data)

#Divide data into 5 classes

In [ ]:
class_0=train_data[train_data[187]==0.0]
class_1=train_data[train_data[187]==1.0]
class_2=train_data[train_data[187]==2.0]
class_3=train_data[train_data[187]==3.0]
class_4=train_data[train_data[187]==4.0]
train_data = pd.concat([class_1, class_2, class_3, class_4, class_0])


In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize = (20,8))
plt.plot(train_data.iloc[0, 0:187], color = 'red')
plt.xlabel("Time (in ms)")
plt.ylabel("Heart Beat Amplitude")
plt.legend()
plt.show()

In [ ]:
from sklearn.utils import resample

class_1_upsample = resample(class_1, n_samples = 20000, replace = True, random_state = 123)
class_2_upsample = resample(class_2, n_samples = 20000, replace = True, random_state = 123)
class_3_upsample = resample(class_3, n_samples = 20000, replace = True, random_state = 123)
class_4_upsample = resample(class_4, n_samples = 20000, replace = True, random_state = 123)

In [ ]:
class_0_downsample = resample(class_0, n_samples = 20000, replace = True, random_state = 123)

In [ ]:
train_data = pd.concat([class_1_upsample, class_2_upsample, class_3_upsample, class_4_upsample, class_0_downsample])

In [ ]:
plt.figure(figsize= (10,10))
my_circle = plt.Circle((0,0), 0.7, color = 'white') 
plt.pie(train_data[187].value_counts(), labels=['Normal Beats','Unknown Beats','Ventricular ectopic beats','Supraventricular ectopic beats',
                                                'Fusion Beats'], autopct = '%0.0f%%', colors = ['red','orange','blue','magenta','cyan'])
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

#  Apply Keras Models

In [ ]:
from keras.utils import to_categorical
Y=train_data.values[:, -1].astype(int)
y=to_categorical(Y)
X_train, X_valid, y_train, y_valid = train_test_split(train_data.drop([187], axis=1), y, test_size = 0.3)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(187,)))
model.add(Dense(50, activation='relu'))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history=model.fit(X_train, y_train, epochs=20,validation_data=(X_valid, y_valid))

print("Evaluation: ")
mse, acc = model.evaluate(X_valid, y_valid)
print('mean_squared_error :', mse)
print('accuracy:', acc)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

In [ ]:
#Performing Classfication on image dataset